In [1]:
frappePath = "/home/mert/frappe/"
frappeRDDraw = sc.textFile(frappePath + "frappe.csv")
frappeRDDraw = frappeRDDraw.map(lambda x: x.split("\t"))
frappeRDDraw.take(2)

[[u'user',
  u'item',
  u'cnt',
  u'daytime',
  u'weekday',
  u'isweekend',
  u'homework',
  u'cost',
  u'weather',
  u'country',
  u'city'],
 [u'0',
  u'0',
  u'1',
  u'morning',
  u'sunday',
  u'weekend',
  u'unknown',
  u'free',
  u'sunny',
  u'United States',
  u'0']]

In [2]:
header = frappeRDDraw.first() #extract header
frappeRDD = frappeRDDraw.filter(lambda x:x !=header) 

#parse and cast
def parseAndCast(line):
    return [int(line[0]),int(line[1]),int(line[2]),line[3],line[4],
            line[5],line[6],line[7],line[8],line[9],int(line[10])]
frappeRDD = frappeRDD.map(parseAndCast)
frappeRDD.take(2)

[[0,
  0,
  1,
  u'morning',
  u'sunday',
  u'weekend',
  u'unknown',
  u'free',
  u'sunny',
  u'United States',
  0],
 [1,
  1,
  7,
  u'afternoon',
  u'saturday',
  u'weekend',
  u'unknown',
  u'free',
  u'cloudy',
  u'Spain',
  0]]

## Group data on user_id

In [3]:
frappeGroupedRDD = frappeRDD.groupBy(lambda x: x[0]).map(lambda x: (x[0], list(x[1]) ))
frappeGroupedRDD.take(2)

[(0,
  [[0,
    0,
    1,
    u'morning',
    u'sunday',
    u'weekend',
    u'unknown',
    u'free',
    u'sunny',
    u'United States',
    0],
   [0,
    26,
    13,
    u'morning',
    u'sunday',
    u'weekend',
    u'unknown',
    u'free',
    u'sunny',
    u'United States',
    0],
   [0,
    21,
    1,
    u'sunset',
    u'tuesday',
    u'workday',
    u'unknown',
    u'free',
    u'cloudy',
    u'United States',
    0],
   [0,
    9,
    9,
    u'evening',
    u'friday',
    u'weekend',
    u'unknown',
    u'free',
    u'cloudy',
    u'United States',
    1023],
   [0,
    2272,
    1,
    u'sunset',
    u'tuesday',
    u'workday',
    u'unknown',
    u'free',
    u'sunny',
    u'United States',
    0],
   [0,
    26,
    2,
    u'evening',
    u'saturday',
    u'weekend',
    u'unknown',
    u'free',
    u'sunny',
    u'United States',
    0],
   [0,
    25,
    1,
    u'evening',
    u'friday',
    u'weekend',
    u'unknown',
    u'free',
    u'cloudy',
    u'United States',


In [4]:
from collections import namedtuple
TrainRow = namedtuple("train", ["itemId", "context"])
ContextRow = namedtuple("context", ["cnt","daytime", "weekday", "isweekend",
                                    "homework", "cost", "weather","country","city"])
def create_objects(line):
    uid = line[0]
    contexts = line[1]
    data = []
    for context in contexts:
        data.append(TrainRow(context[1], ContextRow(*context[2:])))
    l = len(data)
    l1 = int(l*0.8)
    return uid, [data[:l1], data[l1:]]
    
frappeParsedRDD = frappeGroupedRDD.map(create_objects)
frappeParsedRDD.take(5)

[(0,
  [[train(itemId=0, context=context(cnt=1, daytime=u'morning', weekday=u'sunday', isweekend=u'weekend', homework=u'unknown', cost=u'free', weather=u'sunny', country=u'United States', city=0)),
    train(itemId=26, context=context(cnt=13, daytime=u'morning', weekday=u'sunday', isweekend=u'weekend', homework=u'unknown', cost=u'free', weather=u'sunny', country=u'United States', city=0)),
    train(itemId=21, context=context(cnt=1, daytime=u'sunset', weekday=u'tuesday', isweekend=u'workday', homework=u'unknown', cost=u'free', weather=u'cloudy', country=u'United States', city=0)),
    train(itemId=9, context=context(cnt=9, daytime=u'evening', weekday=u'friday', isweekend=u'weekend', homework=u'unknown', cost=u'free', weather=u'cloudy', country=u'United States', city=1023)),
    train(itemId=2272, context=context(cnt=1, daytime=u'sunset', weekday=u'tuesday', isweekend=u'workday', homework=u'unknown', cost=u'free', weather=u'sunny', country=u'United States', city=0)),
    train(itemId=26

In [5]:
frappeParsedRDD.count()

957

In [11]:
from operator import add
numRecList = 5
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output
def topFiveSortedList(oldList, context, probability):
    context_oldList = [x[0] for x in oldList]
    if context not in context_oldList:
        for x in range(0,len(oldList)-1):
            if x == 0 and oldList[x][1] < probability:
                oldList[x] = (context,probability)
                if oldList[x+1][1] < oldList[x][1]:
                    temp = oldList[x+1] 
                    oldList[x+1] = oldList[x]
                    oldList[x] = temp 
                else:
                    break
            elif oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
    else:
        index = context_oldList.index(context)
        if oldList[index][1] < probability:
            oldList[index] = (context,probability)
        oldList = sorted(oldList,key=lambda x: x[1])
    return oldList     

def evaluation_rr(testList, recList):
    scores = 0
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                scores = scores + 1.0 / (i+1)
                break
    if len(testList) != 0 :
        scores = scores / len(testList)
    return scores
def evaluation_rr1(testItem, recList):
    scores = 0
    for i in range(len(recList)):
        if testItem == recList[i]:
            scores = 1.0 / (i+1)
            break
    return scores

def evaluation_precision(testItem, recList):
    scores = 0
    if testItem in recList:
        scores = 1.0
    return scores    
    
def evaluation_rap(testList,recList):
    scores = 0.0
    hitNum = 0  
    for i in range(len(recList)):
        if recList[i] in testList:
            hitNum = hitNum + 1
            scores = scores + 1.0*hitNum/(i+1)
    if hitNum != 0:
        scores = scores/hitNum
    else:
        scores = 0
    return scores
        

def evaluation_ap(testList, recList):
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    if rr[j] == 1:
                        scores = scores + 1.0 / (j+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores

def evaluation_recall(testList, recList):
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(len(recList))]
    rr=[]
    rr[:len(recList)] = [0 for i in range(len(recList))]
    for t in testList:
        for i in range(len(recList)):
            if t == recList[i]:
                rr[i] = 1
    for t in testList:    
        for i in range(len(recList)):
            if t == recList[i]:
                for j in range(i,len(recList)):    
                    recall[j] = recall[j] + 1.0 
                break    
    recall = [0 if len(testList) == 0 else x/len(testList) for x in recall]
    return recall
def print_result2(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation1.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
    f.close()
    
def print_result1(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation1.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by precision:'+str(finalScore.mean()))
    f.write('\n')
    f.close()
def print_result2(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation3.txt','a')
    f.write('2000 users with duplicated evaluations')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[3]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.close()
def print_result4(finalScore, algorithm_name):
    f = open('result/frappe1.txt','a')
    f.write('all(957) users from frappe:')
    f.write('\n')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by mrr1:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[2]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[3][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[3][i]).mean()))
            
    f.write(algorithm_name + ' by precision:'+str(finalScore.map(lambda x:x[4]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by rap:'+str(finalScore.map(lambda x:x[5]).mean()))
    f.write('\n')
    f.close()

In [20]:
from collections import Counter
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = frappeParsedRDD.map(mfuFunction)
print_result2(finalScore,'mfu')

In [24]:
def bayesian_context_daytime(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][2]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][2]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
#    return line[2][1]
finalScore = frappeParsedRDD.map(bayesian_context_daytime)
#finalScore.take(1)
print_result2(finalScore, 'bayesian_context_daytime')

In [25]:
def bayesian_context_weekday(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][3]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][3]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
#    return line[2][1]
finalScore = frappeParsedRDD.map(bayesian_context_weekday)
#finalScore.take(1)
print_result2(finalScore, 'bayesian_context_weekday')

In [26]:
def bayesian_context_isweekend(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
#    return line[2][1]
finalScore = frappeParsedRDD.map(bayesian_context_isweekend)
#finalScore.take(1)
print_result2(finalScore, 'bayesian_context_isweekend')

In [29]:
def bayesian_context_homework(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][5]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][5]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
#    return line[2][1]
finalScore = frappeParsedRDD.map(bayesian_context_homework)
#finalScore.take(1)
print_result2(finalScore, 'bayesian_context_homework')

In [28]:
def bayesian_context_weather(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][7]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][7]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_ap, scores_recall, scores_precision, scores_rap
#    return line[2][1]
finalScore = frappeParsedRDD.map(bayesian_context_weather)
#finalScore.take(1)
print_result2(finalScore, 'bayesian_context_weather')

In [12]:
from collections import Counter
def mfuFunction(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
        Recommender =  [t[0] for t in Recommender] 

        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr1 = scores_rr1 / numTest
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = frappeParsedRDD.map(mfuFunction)
print_result4(finalScore,'mfu')

In [15]:
def bayesian_context_homework(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][5]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][5]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = len(testList)
    
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_precision = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):
        context = [x for x in trainList if x[1]==testList[0][1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app_context = float(len(contextAppi))/len(appi)
            if p_app_context>0.3:
                p_app = topFiveSortedList(p_app,c[0],p*p_app_context*5)
            else:
                p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        testList_items = [t[0] for t in testList]                      #take only id for test set
        finalRecommender =  map(lambda x:x[0],p_app[:5])        #take only id for train set

        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(lambda x,y :x+y, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        
        trainList.append(testList[0])
        
        testList.pop(0)  
    if numTest != 0:
        scores_rr1 = scores_rr1 / numTest
        scores_rr = scores_rr / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr,scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
finalScore = frappeParsedRDD.map(bayesian_context_homework)
print_result4(finalScore, 'bayesian_context_homework')

In [16]:
from operator import add
from collections import Counter

def mfu_mru(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        #Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender = Counter(RecommenderDuplicate).most_common()
#        testcount = Counter(testList_items).most_common()
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        RecommenderDuplicate1 =  [t[0] for t in trainList]        #take only id for train set
        Recommender1 = remove_duplicates(RecommenderDuplicate1)    #remove duplicate
        Recommender = [[t[0],t[1]] for t in Recommender if t[0] in Recommender1[:10]]
#         Recommender = [[t[0],1.0*t[1]/numTrain] for t in Recommender]
#         testcount = [[t[0],1.0*t[1]/numTest] for t in testcount]
       
    #weighted
#         Recommender =  [[t,0] for t in Recommender] 
#         for i in range(len(Recommender)):
#             for j in range(len(trainList)):
#                 if trainList[j][0] == Recommender[i][0]:
#                     if len(trainList)>0 and (trainList[-1][1][0] - trainList[j][1][0]) == 0:
#                         weight = 1
#                     elif len(trainList)>0:
#                         weight = (trainList[-1][1][0] - trainList[j][1][0])/86400
#                     else:
#                         weight = 0
#                     if weight != 0:
#                         Recommender[i][1] = Recommender[i][1] + 1.0/weight
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
    #return Recommender[:10],numTrain,testcount,numTest

finalScore = frappeParsedRDD.map(mfu_mru)
print_result4(finalScore,'mfu_mru')

In [17]:
from operator import add
from collections import Counter

def mfu_decay(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender =  [[t,0] for t in Recommender] 
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        decay = 1
        for i in range(len(trainList)):
            if decay>0:
                index = [x[0] for x in Recommender].index(trainList[i][0])
                Recommender[index][1] = Recommender[index][1] + decay
            else:
                break
            decay = decay - 1.0/numTrain #0.02 is worse
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
    #return Recommender[:10],numTrain,testcount,numTest

finalScore = frappeParsedRDD.map(mfu_decay)
print_result4(finalScore,'mfu_decay')

In [18]:
from operator import add
from collections import Counter

def mfu_decay2(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender =  [[t,0] for t in Recommender] 
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        decay = 1
        for i in range(len(trainList)):
            index = [x[0] for x in Recommender].index(trainList[i][0])
            Recommender[index][1] = Recommender[index][1] + decay
            decay = decay * 0.99 #0.9 is worse
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
    #return Recommender[:10],numTrain,testcount,numTest

finalScore = frappeParsedRDD.map(mfu_decay2)
print_result4(finalScore,'mfu_decay2')

In [19]:
from operator import add
from collections import Counter

def mfu_mru_decay2(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    numTrain = len(trainList)
    numTest = len(testList)
    scores_precision = 0.0
    scores_rr = 0.0
    scores_rr1 = 0.0
    scores_ap = 0.0
    scores_rap = 0.0
    scores_recall = [0.0, 0.0, 0.0, 0.0, 0.0]
    for i in range(numTest):  
        testList_items = [t[0] for t in testList]                #take only id for test set
        RecommenderDuplicate =  [t[0] for t in trainList]        #take only id for train set
        Recommender = remove_duplicates(RecommenderDuplicate)    #remove duplicate
        Recommender =  [[t,0] for t in Recommender] 
        
        trainList = sorted(trainList,key=lambda x:int(x[1][0]), reverse=True);  #sort by timestamp with descending
        RecommenderDuplicate1 =  [t[0] for t in trainList]        #take only id for train set
        Recommender1 = remove_duplicates(RecommenderDuplicate1)    #remove duplicate
        decay = 1
        for i in range(len(trainList)):
            index = [x[0] for x in Recommender].index(trainList[i][0])
            Recommender[index][1] = Recommender[index][1] + decay
            decay = decay * 0.99
        Recommender = [[t[0],t[1]] for t in Recommender if t[0] in Recommender1[:10]]
        Recommender = sorted(Recommender,key=lambda x:-x[1])
        Recommender =  [t[0] for t in Recommender] 
        if len(Recommender) > 4:
            finalRecommender = Recommender[:5]
        else:
            finalRecommender = [-1,-1,-1,-1,-1]
            numRec = len(Recommender)
            finalRecommender[:numRec] = Recommender
        scores_rr = evaluation_rr(testList_items, finalRecommender) + scores_rr
        scores_rr1 = evaluation_rr1(testList[0][0], finalRecommender) + scores_rr1
        scores_ap = evaluation_ap(testList_items, finalRecommender) + scores_ap
        scores_recall = map(add, evaluation_recall(testList_items, finalRecommender), scores_recall)
        scores_precision = evaluation_precision(testList[0][0], finalRecommender) + scores_precision
        scores_rap = evaluation_rap(remove_duplicates(testList_items), finalRecommender) + scores_rap
        trainList = sorted(trainList,key=lambda x:int(x[1][0])); ####
        trainList.append(testList[0])
        testList.pop(0)  
    if numTest != 0:
        scores_rr = scores_rr / numTest
        scores_rr1 = scores_rr1 / numTest
        scores_ap = scores_ap / numTest
        scores_recall = [x / numTest for x in scores_recall]
        scores_precision = scores_precision / numTest
        scores_rap = scores_rap / numTest
    return scores_rr, scores_rr1, scores_ap, scores_recall, scores_precision, scores_rap
    #return Recommender[:10],numTrain,testcount,numTest

finalScore = frappeParsedRDD.map(mfu_mru_decay2)
print_result4(finalScore,'mfu_mru_decay2')